In [1]:
from sentence_transformers import SentenceTransformer, util
sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

x = util.pytorch_cos_sim(embedding_1, embedding_2)
print(x)

c:\Users\amir.rahman\AppData\Local\anaconda3\envs\semantic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[0.6003]])


In [4]:
import pandas as pd

raw = pd.read_csv('cordlife_faqs1.csv')

lookup = {}
for a, b, *_ in raw.values:
    lookup[a] = b

raw

,prompt,response,Unnamed: 2
0,What is cord blood?,Cord blood is blood that remains in the umbili...,NaN
1,What are cord blood stem cells?,Cord blood stem cells are also known as Haemat...,NaN
2,Why should I save my baby's cord blood stem ce...,There are several advantages of storing your b...,NaN
3,What can cord blood stem cells do?,Cord blood stem cells can:\n\nReplace and rege...,NaN
4,How does a cord blood stem cell transplant work?,The purpose of a stem cell treatment is to rec...,NaN
...,...,...,...
58,When will the new GST change from 7% to 8% tak...,The new GST change from 7% to 8% will take eff...,NaN
59,How will the GST change affect my payments to ...,For Existing Clients\nThe revision in GST rate...,NaN
60,"With the absorption of the 1% GST increase, wh...",Any enrolment on or before 31 December 2022 wi...,NaN
61,What should I do if I am currently on an annua...,You can consider upgrading your current annual...,NaN


In [5]:
from sentence_transformers import SentenceTransformer, util

sentences = list(raw['prompt'].values)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentences_embeddings = [model.encode(s, convert_to_tensor=True) for s in sentences]


In [6]:
def search(q, THRESHOLD=0.65):
    q_embedding = model.encode(q, convert_to_tensor=True)
    out = []
    for sentence, sentence_embedding in zip(sentences, sentences_embeddings):
        score = util.pytorch_cos_sim(q_embedding, sentence_embedding).item()
        out.append((sentence, score))
    out.sort(key=lambda x:-x[-1])
    if out[0][1] < THRESHOLD:
        return ['I don\'t know']
    top = out[:3]
    return [(qns, lookup[qns], score) for qns,score in top]

search('what is cordlife preferential plan and how would it benefit a parent like me')

[('How would Preferential Plan benefit a Cordlife Parent like me?',
  "As a gesture of our appreciation to our Cordlife parents, we've introduced the Preferential Plan which allows Cordlife Preferential Plan lets you enjoy: Protection against increases in Annual Instalment Fees. Protection against hikes in Goods and Services Tax (GST). The convenience of a one-time payment for the remaining balance of annual instalment fee. Multiple payment modes such as Child Development Account (CDA), credit card interest-free instalment plans & etc.                                                           Contact us to find out more about the full benefits.",
  0.9085880517959595),
 ('What is Cordlife Preferential Plan?',
  "As a gesture of our appreciation to our Cordlife parents, we've introduced the Preferential Plan which allows parents to prepay the balance of their annual fees. Multiple payment modes are available; including the use of your child's Child Development Account (CDA) for this one

In [7]:
search('what is CDA and how does it work')

[('What is CDA First Step Grant?',
  'The CDA First Step Grant is new grant of $3,000 for eligible Singaporean children born from 24 March 2016. It is paid into the child�s CDA and forms part of the existing overall Government contribution cap.\n\nSingaporean children born from 24 March 2016 who are Singaporean Citizens (or become a citizen before turning 12 years old) and have lawfully married parents, are eligible for the CDA benefits.',
  0.7030069828033447),
 ('How does the Child Development Account (CDA) work?',
  'The CDA has two components: the CDA First Step Grant and the Government Dollar-for-Dollar Matching.\n\nUnder the CDA First Step Grant, parents will receive an initial amount of $3,000 (from the Government�s existing contribution caps) which will be deposited into your child�s account.',
  0.634533703327179),
 ('What is a Child Development Account (CDA)?',
  'The CDA* is a special savings account for children to help build up the savings that can be spent on approved use

In [8]:
search('does cordlife plan to launch anything in the near future')

["I don't know"]

In [9]:
search('does cordlife have branches in europe or america')

["I don't know"]

In [10]:
search('can i donate blood to cordlife')

["I don't know"]